In [1]:
import numpy as np
from bs4 import BeautifulSoup as BS
import requests
import pandas as pd
import sys

In [23]:
url = 'https://www.eia.gov/dnav/ng/hist/rngwhhdW.htm'
html = requests.get(url).text
soup = BS(html)
rows = soup.select('table[SUMMARY] tr')
len(rows)

300

In [78]:
df = pd.DataFrame(rows[2:], columns=['tag'])
df.info()
df.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 298 entries, 0 to 297
Data columns (total 1 columns):
tag    298 non-null object
dtypes: object(1)
memory usage: 2.4+ KB


,tag
0,"<tr> <td class=""B6""> 1997-Jan</td> <td class=..."
1,"<tr> <td class=""B6""> 1997-Feb</td> <td class=..."
2,"<tr> <td class=""B6""> 1997-Mar</td> <td class=..."
3,"<tr> <td class=""B6""> 1997-Apr</td> <td class=..."
4,"<tr> <td class=""B6""> 1997-May</td> <td class=..."


In [80]:
df['year_month'] = df['tag'].apply(lambda x: x.select_one('.B6'))
df.dropna(inplace=True)
df.info()
df.head()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 272 entries, 0 to 293
Data columns (total 2 columns):
tag           272 non-null object
year_month    272 non-null object
dtypes: object(2)
memory usage: 6.4+ KB


,tag,year_month
0,"<tr> <td class=""B6""> 1997-Jan</td> <td class=...","<td class=""B6""> 1997-Jan</td>"
1,"<tr> <td class=""B6""> 1997-Feb</td> <td class=...","<td class=""B6""> 1997-Feb</td>"
2,"<tr> <td class=""B6""> 1997-Mar</td> <td class=...","<td class=""B6""> 1997-Mar</td>"
3,"<tr> <td class=""B6""> 1997-Apr</td> <td class=...","<td class=""B6""> 1997-Apr</td>"
4,"<tr> <td class=""B6""> 1997-May</td> <td class=...","<td class=""B6""> 1997-May</td>"


In [104]:
data = {}

def unpack_data(tag):
    strings = (i.strip() for i in tag.strings if i.strip())
    year = next(strings).split('-')[0]
    while True :
        try:
            date = next(strings)
            data[f'{year} {date}'] = [next(strings)]
        except StopIteration:
            break

df['tag'].apply(lambda x: unpack_data(x))
ddf = pd.DataFrame(data).transpose()
ddf.info()
ddf.head()

<class 'pandas.core.frame.DataFrame'>
Index: 1180 entries, 1997 01/10 to 2019 08/23
Data columns (total 1 columns):
0    1180 non-null object
dtypes: object(1)
memory usage: 18.4+ KB


,0
1997 01/10,3.79
1997 01/17,4.19
1997 01/24,2.98
1997 01/31,2.91
1997 02/07,2.53


In [105]:
ddf.index = pd.to_datetime(ddf.index)

In [106]:
ddf.to_csv('test.csv', header=False)